In [165]:
import pandas as pd
from bokeh.plotting import figure 
from bokeh.io import output_file, save, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, \
    HoverTool, LinearAxis, NumeralTickFormatter, Range1d, RangeTool

In [166]:
# Enable viewing Bokeh plots in the notebook
from bokeh.io import show, output_notebook
output_notebook()

In [167]:
# For google collab to wrap output
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [168]:
# Task 1: Prepare the Data
stock_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTiM1scE44za7xyuheW_FrUkdSdOKipDgDOWa_03ixmJCWK_ReSqhjzax66nNHyDKARXWIXgFI_EW9X/pub?gid=1661368486&single=true&output=csv'
stock = pd.read_csv(stock_url)

metrics_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRDaf4y17OWjQqxODuxA4q4hsvXRkSqN0na1KtTIpvOZUdc7xHbrkhcygFfDIyVQWI2UbC3YcKUbser/pub?gid=981872466&single=true&output=csv'
metrics = pd.read_csv(metrics_url)

In [169]:
stock

,Symbol,Date,Open,High,Low,Close,Adj Close,Volume
0,META,1/1/2019,128.990005,138.869995,128.559998,138.050003,138.050003,99955500
1,META,1/8/2019,139.889999,146.570007,139.539993,145.389999,145.389999,98023000
2,META,1/15/2019,146.009995,152.429993,145.990005,150.039993,150.039993,88912200
3,META,1/22/2019,149.199997,151.529999,142.520004,147.470001,147.470001,101178300
4,META,1/29/2019,148.089996,171.679993,143.429993,169.250000,169.250000,190321400
...,...,...,...,...,...,...,...,...
1065,AAPL,1/3/2023,130.279999,133.410004,124.169998,130.149994,129.951584,440671200
1066,AAPL,1/10/2023,130.259995,134.919998,128.119995,134.759995,134.554550,262492700
1067,AAPL,1/17/2023,134.830002,143.320007,133.770004,141.110001,140.894882,353332300
1068,AAPL,1/24/2023,140.309998,147.229996,138.809998,143.000000,142.781998,320847600


In [170]:
metrics

,Symbol,Quarter Ended,Revenue Growth (YoY),EPS Growth,PE Ratio
0,META,2019-03-31,0.2600,-0.4970,24.33
1,META,2019-06-30,0.2762,-0.4770,32.28
2,META,2019-09-30,0.2859,0.2045,28.20
3,META,2019-12-31,0.2464,0.0802,31.67
4,META,2020-03-31,0.1764,1.0118,22.69
...,...,...,...,...,...
75,AAPL,2021-12-31,0.1122,0.2500,28.76
76,AAPL,2022-03-31,0.0859,0.0857,27.97
77,AAPL,2022-06-30,0.0187,-0.0769,23.01
78,AAPL,2022-09-30,0.0814,0.0488,24.22


In [171]:
## 1.1: Convert the data type of time columns to datetime using to_datatime()
stock['Date'] = pd.to_datetime(stock['Date'])
metrics['Quarter Ended'] = pd.to_datetime(metrics['Quarter Ended'])

In [172]:
# Define a function that create a candlestick chart for a company 
def create_candlestick_chart(symbol):
    
    ## 2.1 Create the data source and set the basic properties of the figure

    # Use ColumnDataSource to create the data source
    # The dataframe is not directly used as source here
    # because you'll use CDSView filter later
    # which works with ColumnDataSource
    data = stock[stock.Symbol == symbol]
    source = ColumnDataSource(data)
    
    
    p = figure(
        width=800, 
        height=400, 
        title=symbol, 
        # Specify the x range to be (min date, max date)
        # so that you can refer to this range in other plots
        x_range=(data["Date"].min(), data["Date"].max()),            
        # Set the x axis to show date time
        x_axis_type="datetime", 
        # Put the x axis to be at the top of the plot
        x_axis_location= "above",
        background_fill_color = '#fbfbfb',
        tools="pan,wheel_zoom,reset", 
        toolbar_location='right',
    )
    
    p.xgrid.grid_line_color='#e5e5e5'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.major_label_text_font_size = '10px'
    p.yaxis.axis_label = 'Stock Price in USD'
    p.yaxis.formatter = NumeralTickFormatter(format="($ 0,0 a)")
    
    ## 2.2: Manually set the y axis range 
    
    # Make it larger than the (min, max) range of the data
    # e.g. (min * 0.9, max * 1.1)
    # https://docs.bokeh.org/en/3.0.3/docs/reference/models/ranges.html#bokeh.models.DataRange1d
    p.y_range.start = data.Low.min() * 0.9
    p.y_range.end = data.High.max() * 1.1
    
    ## 2.3: Use CDSView to create two filters on the stock data
    
    # 'inc' keeps the data where the close price is higher than the open price
    # 'dec' does the opposite of 'inc' 
    # https://docs.bokeh.org/en/latest/docs/user_guide/basic/data.html#filtering-data

    inc = source.data['Close'] > source.data['Open']
    dec = source.data['Close'] < source.data['Open']
    
    inc_view = CDSView(source=source, filters=[BooleanFilter(inc)])
    dec_view =  CDSView(source=source, filters=[BooleanFilter(dec)])

    ## 2.4: Draw the glyphs in the candlesticks

    # A candlestick consists of a segment and a vbar
    
    # Set the width of the vbar
    # Note that the unit of datetime axis is milliseconds
    # while the interval of the stock data is week
    # w = 6.048e+8 # 7 Days in ms
    w = 5.616e+8 # 6.5 Days in ms for a cleaner look. Candles are separate when you zoom in.
    
    # Draw the segement and the vbar
    # inc vbars are green, dec vbars are red
    stock_segment = p.segment(x0='Date', x1='Date', y0='High', y1='Low', width=2, color="black", source=source)

    stock_inc = p.vbar(x='Date', width=w, top='Open', bottom='Close', fill_color="green", line_color="green", name="price", source=source,view=inc_view)

    stock_dec = p.vbar(x='Date', width=w, top='Open', bottom='Close', fill_color="red", line_color="red", name="price", source=source,view=dec_view)


    ## 2.5: Add volume bars in the candlestick chart
    
    # Note that volume data is different from stock price data in unit and scale
    # You'll create a twin y axis for volume on the right side of the chart
    # https://docs.bokeh.org/en/3.0.2/docs/user_guide/basic/axes.html#twin-axes
    # https://docs.bokeh.org/en/latest/docs/reference/models/axes.html#bokeh.models.LinearAxis

    y_volume = data.Volume
    p.extra_y_ranges['volume'] =  Range1d(start=y_volume.min(), end=y_volume.max())
    
    # TODO
    y_volume_axis =  LinearAxis(
        y_range_name="volume",
        axis_label="Volume",
        formatter=NumeralTickFormatter(format="0a"),
    )

    
    p.add_layout(y_volume_axis, 'right')
    
    stock_volume = p.vbar(
         x="Date",
        top="Volume",
        y_range_name="volume",
        source=source,
        width=0.5,
        alpha=0.2,
        line_color="grey",
    )
    
    ## 2.6: Add a hover tool for the candlesticks

    # https://stackoverflow.com/questions/61175554/how-to-get-bokeh-hovertool-working-for-candlesticks-chart
    hover_stock = HoverTool()
    hover_stock.tooltips = [
        ("Date", "@Date{%Y-%m-%d}"),
        ("Open", "@Open{($0,0)}"),
        ("Close", "@Close{($0,0)}"),
        ("High", "@High{($0,0)}"),
        ("Low", "@Low{($0,0)}"),
        ("Volume", "@Volume{($0,0 a)}"),
    ]
    hover_stock.formatters = {
        "@Date": "datetime",
    }

    # This hover tool only shows tooltips on the vbars in the candlesticks, thus you need to specify the renderers
    hover_stock.renderers = [
        stock_inc, stock_dec
    ]
    p.add_tools(hover_stock)
    
    p.output_backend = 'svg'
    
    return p

In [173]:
# Task 3: Add Metrics Plot to the Candlestick Chart

def add_metrics_plot(main_plot):
    
    p = main_plot
    symbol = p.title.text

    # See how bokeh deals with data source containing nan values
    # https://docs.bokeh.org/en/latest/docs/user_guide/basic/lines.html#missing-points
    # Note that this might not work if the source is created from ColumnDataSource
    data = metrics[metrics.Symbol == symbol]
    source = ColumnDataSource(data)
    
    ## 3.1: Set the y axes for the metrics

    # 'PE Ration' and 'EPS Growth' also have different units from the stock price
    # You'll set a y axis for each metric
    # and make the extra y axes invisible
    y_pe = source.data['PE Ratio']
    y_eps = source.data['EPS Growth']
    
    p.extra_y_ranges['pe'] = Range1d(y_pe.min() * 0.9, y_pe.max() * 1.1)
    p.extra_y_ranges['eps'] = Range1d(y_eps.min() * 0.9, y_eps.max() * 1.1)

    
    y_pe_axis = LinearAxis( y_range_name="pe",
        visible=False)

    y_eps_axis = LinearAxis( y_range_name="eps",
        visible=False)
    
    ## 3.2 Use scatter and line glyphs to plot the metrics
    pe_l = p.line(
      source=source,
      y_range_name="pe",
      x="Quarter Ended",
      y="PE Ratio",
      legend_label = "PE Ratio",
        line_color="grey",
    )
    
    pe_c = p.circle(
 source=source,
      y_range_name="pe",
      x="Quarter Ended",
      y="PE Ratio",
      legend_label = "PE Ratio",
      line_color="grey", fill_color="white", line_width=1
    )
    
    eps_l = p.line(
      source=source,
      y_range_name="eps",
      x="Quarter Ended",
      y="EPS Growth",
      legend_label = "EPS Growth",
        line_color="grey",

    )
    
    eps_c = p.circle(
        source=source,
      y_range_name="eps",
      x="Quarter Ended",
      y="EPS Growth",
      legend_label = "EPS Growth",
      line_color="grey", fill_color="grey", line_width=1
    )
     ## 3.3: Make the legend of metrics interactive
    
    # Click on a legend to mute the corresponding glyph(s)
    # https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html#hiding-glyphs
    p.legend.click_policy= "mute"

    # Set the attributes of the legend to adjust its postion, color and size
    # https://docs.bokeh.org/en/latest/docs/reference/models/annotations.html#bokeh.models.Legend
    p.legend.location = 'top_left'
    p.legend.orientation = 'horizontal'
    p.legend.background_fill_alpha = 0
    p.legend.border_line_alpha = 0
    p.legend.label_text_font_size = '10px'
    p.legend.glyph_width = 16
    
    ## 3.4: Add a hovertool for the scatter glyphs

    metrics_hover = HoverTool()
    metrics_hover.tooltips=[
        ("Quarter Ended", "@{Quarter Ended}{%Y-%m-%d}"),
        ("PE Ratio", "@{PE Ratio}"),
        ("EPS Growth", "@{EPS Growth}"),
    ]
    metrics_hover.formatters={
       "@{Quarter Ended}": "datetime",
    }
    metrics_hover.mode='mouse' 
    metrics_hover.renderers = [pe_c, eps_c]                
    p.add_tools(metrics_hover) 
    
    return p


In [174]:
if __name__ == "__main__":
  p = create_candlestick_chart('AAPL')
  p = add_metrics_plot(p)
  #p = add_select_range(p)
#output_file('dvc_ex2.html')
#save(p)
  show(p)